# Census aggregation scratchpad

By [Ben Welsh](https://palewi.re/who-is-ben-welsh/)

In [119]:
import math

### Approximation

![](https://assets.documentcloud.org/documents/6162551/pages/20180418-MOE-p50-normal.gif)
![](https://assets.documentcloud.org/documents/6162551/pages/20180418-MOE-p51-normal.gif)

In [120]:
males_under_5, males_under_5_moe = 10154024, 3778

In [121]:
females_under_5, females_under_5_moe = 9712936, 3911

In [122]:
total_under_5 = males_under_5 + females_under_5

In [123]:
total_under_5

19866960

In [124]:
total_under_5_moe = math.sqrt(males_under_5_moe**2 + females_under_5_moe**2)

In [125]:
total_under_5_moe

5437.757350231803

![](https://assets.documentcloud.org/documents/6162551/pages/20180418-MOE-p52-normal.gif?1561126109)

In [126]:
def approximate_margin_of_error(*pairs):
    """
    Returns the approximate margin of error after combining all of the provided Census Bureau estimates, taking into account each value's margin of error.
    
    Expects a series of arguments, each a paired list with the estimated value first and the margin of error second.
    """
    # According to the Census Bureau, when approximating a sum use only the largest zero estimate margin of error, once
    # https://www.documentcloud.org/documents/6162551-20180418-MOE.html#document/p52
    zeros = [p for p in pairs if p[0] == 0]
    if len(zeros) > 1:
        max_zero_margin = max([p[1] for p in zeros])
        not_zero_margins = [p[1] for p in pairs if p[0] != 0]
        margins = [max_zero_margin] + not_zero_margins
    else:
        margins = [p[1] for p in pairs]
    return math.sqrt(sum([m**2 for m in margins]))    

In [127]:
approximate_margin_of_error(
    (males_under_5, males_under_5_moe),
    (females_under_5, females_under_5_moe)
)

5437.757350231803

In [128]:
approximate_margin_of_error(
    [0, 22],
    [0, 22],
    [0, 29],
    [41, 37]
)

47.01063709417264

### Aggregating totals

In [129]:
def total(*pairs):
    """
    Returns the combined value of all the provided Census Bureau estimates, along with an approximated margin of error.
    
    Expects a series of arguments, each a paired list with the estimated value first and the margin of error second.
    """
    return sum([p[0] for p in pairs]), approximate_margin_of_error(*pairs)

In [130]:
total(
    (males_under_5, males_under_5_moe),
    (females_under_5, females_under_5_moe)
)

(19866960, 5437.757350231803)

In [131]:
total(
    [0, 22],
    [0, 22],
    [0, 29],
    [41, 37]
)

(41, 47.01063709417264)

### Aggregating medians

![](https://assets.documentcloud.org/documents/6165014/pages/How-to-Recalculate-a-Median-p1-normal.gif?1561138970)
![](https://assets.documentcloud.org/documents/6165014/pages/How-to-Recalculate-a-Median-p2-normal.gif?1561138970)
![](https://assets.documentcloud.org/documents/6165014/pages/How-to-Recalculate-a-Median-p4-normal.gif?1561138970)

In [150]:
def approximate_median(range_list):
    """
    Returns the estimated median from a set of ranged totals.

    Useful for generated medians for measures like median household income and median agn when aggregating census geographies.

    Expects a list of dictionaries with three keys:

        min: The minimum value in the range
        max: The maximum value in the range
        n: The number of people, households or other universe figure in the range
    """
    # Sort the list
    range_list.sort(key=lambda x: x['min'])

    # What is the total number of observations in the universe?
    n = sum([d['n'] for d in range_list])

    # What is the midpoint of the universe?
    midpoint = n / 2.0

    # For each range calculate its min and max value along the universe's scale
    cumulative_n = 0
    for range_ in range_list:
        range_['n_min'] = cumulative_n
        cumulative_n += range_['n']
        range_['n_max'] = cumulative_n

    # Now use those to determine which group contains the midpoint.
    try:
        midpoint_range = next(d for d in range_list if midpoint >= d['n_min'] and midpoint <= d['n_max'])
    except StopIteration:
        raise StopIteration("The midpoint of the total does not fall within a data range.")

    # How many households in the midrange are needed to reach the midpoint?
    midrange_gap = midpoint - midpoint_range['n_min']

    # What is the proportion of the group that would be needed to get the midpoint?
    midrange_gap_percent = midrange_gap / midpoint_range['n']

    # Apply this proportion to the width of the midrange
    midrange_gap_adjusted = (midrange_gap['max'] - midrange_gap['min']) * midrange_gap_percent

    # Estimate the median
    estimated_median = midrange_gap['min'] + midrange_gap_adjusted

    # Return the result
    return estimated_median

In [151]:
income = [
    dict(min=-2500, max=9999, n=186),
    dict(min=10000, max=14999, n=78),
    dict(min=15000, max=19999, n=98),
    dict(min=20000, max=24999, n=287),
    dict(min=25000, max=29999, n=142),
    dict(min=30000, max=34999, n=90),
    dict(min=35000, max=39999, n=107),
    dict(min=40000, max=44999, n=104),
    dict(min=45000, max=49999, n=178),
    dict(min=50000, max=59999, n=106),
    dict(min=60000, max=74999, n=177),
    dict(min=75000, max=99999, n=262),
    dict(min=100000, max=124999, n=77),
    dict(min=125000, max=149999, n=100),
    dict(min=150000, max=199999, n=58),
    dict(min=200000, max=250001, n=18)
]

In [152]:
approximate_median(income)

TypeError: 'float' object is not subscriptable